In [1]:
using ITensors,ITensorMPS
import ITensors:op
using PyFormattedStrings
using LinearAlgebra
using Plots

ITensors.disable_warn_order()

14

In [2]:
# overloadings for SiteType 3/2
#----------------------------------------------
ITensors.space(::SiteType"S=3/2") = 4

# operator Sz
#-------------------
ITensors.op(::OpName"Sz",::SiteType"S=3/2") =
[+3/2   0    0    0
  0    +1/2  0    0
  0     0   -1/2  0
  0     0    0   -3/2]

# operator S+
#-------------------
ITensors.op(::OpName"S+",::SiteType"S=3/2") =
[0  √3  0   0 
 0  0   √4  0
 0  0   0   √3
 0  0   0   0]

# operator S-
#-------------------
ITensors.op(::OpName"S-",::SiteType"S=3/2") =
[0  0   0   0
 √3 0   0   0
 0  √4  0   0
 0  0   √3  0]

# z projection states
#-----------------------------------------
ITensors.state(::StateName"+3/2",::SiteType"S=3/2") = [1, 0, 0, 0]
ITensors.state(::StateName"+1/2",::SiteType"S=3/2") = [0, 1, 0, 0]
ITensors.state(::StateName"-1/2",::SiteType"S=3/2") = [0, 0, 1, 0]
ITensors.state(::StateName"-3/2",::SiteType"S=3/2") = [0, 0, 0, 1]

In [3]:
# construction of the MPO hamiltonian
#------------------------------------------
function Hamiltonian(sites)
    ops = OpSum()

    N = length(sites)
    # spin-spin interaction
    for k=1:N-1
        ops += J,   "Sz",k, "Sz",k+1
        ops += J/2, "S+",k, "S-",k+1
        ops += J/2, "S-",k, "S+",k+1
    end
    #make it a ring
    ops += J,   "Sz",N, "Sz",1
    ops += J/2, "S+",N, "S-",1
    ops += J/2, "S-",N, "S+",1

    # spin-B interaction
    for k=1:N
        ops += -B, "Sz",k
    end
    
    H = MPO(ops,sites)
    #--------------------------
    return H
end

Hamiltonian (generic function with 1 method)

In [4]:
# apply magon operators at various sites
#----------------------------------
function MagnonOperation(wf::MPS, k_list; split="no")    
    psi = copy(wf)
    sites = siteinds(psi)

    #magnon creation at sites k,k+1
    for k in k_list
        if isodd(k)
            psi = apply( op("S+",sites[k])  , psi )
            psi = apply( op("S-",sites[k+1]), psi )
        else
            psi = apply( op("S-",sites[k])  , psi )
            psi = apply( op("S+",sites[k+1]), psi )
        end   
    end

    #for split magnon
    if length(k_list)==2 && split=="yes"
        m,n = k_list[1]+1, k_list[2]     #loc of desturction
        
        if isodd(m)
            psi = apply( op("S-",sites[m]), psi )
        else
            psi = apply( op("S+",sites[m]), psi )
        end

        if isodd(n)
            psi = apply( op("S-",sites[n]), psi )
        else
            psi = apply( op("S+",sites[n]), psi )
        end
    end

    return normalize!(psi)
    #----------------------------------
end

MagnonOperation (generic function with 1 method)

In [5]:
# one or two sites reduced density matrix
#--------------------------------------
function get_2pdm(wf::MPS,locs)
    ket = copy(wf)
    sites = siteinds(ket)
    n_locs = length(locs)

    if n_locs==1
        A = locs[1]
        orthogonalize!(ket,A)
        rho = prime(ket[A],sites[A]) * dag(ket[A])

    elseif n_locs==2
        A,B = locs
        orthogonalize!(ket,A)
        bra = prime(dag(ket),linkinds(ket))
        
        rho = prime(ket[A],linkinds(ket,A-1)) * prime(bra[A],sites[A])
        for k in A+1:B-1
            rho *= ket[k]*bra[k]
        end
        rho *= prime(ket[B],linkinds(ket,B)) * prime(bra[B],sites[B])  
    
    end

    #-----------------------------------
    return rho
end


#four point RDM of sites AB, CD
# AB adjacent, CD adjacent
#------------------------------------------
function get_4pdm(wf::MPS,locs)
    psi = copy(wf)
    A,C = locs

    orthogonalize!(psi,A)
    ket = psi[A]
    for k in A+1:C+1
        ket *= psi[k]
    end
    rho = prime(ket,"Site") * dag(ket)

    if C-A>2
        inds_list = inds(rho)
        trinds1 = collect(3 : C-A)
        trinds2 = trinds1[end] .+ trinds1 .+ 2
        for idx in 1:length(trinds1)
            rho = rho*delta(inds_list[trinds1[idx]],inds_list[trinds2[idx]])
        end
    end
    #-------------------
    return rho
end

get_4pdm (generic function with 1 method)

In [6]:
#   calc_Eigenvalues of density matrix
#-----------------------------------
function calc_Eigenvalues(density_matrix)
    egn_val,_ = eigen(density_matrix,ishermitian=true)
    #-----------------------------------
    return diag(array(egn_val))
end


#   eigenvalue based generic properties of a density matrix
#-----------------------------------
function calc_Norm(density_matrix)
    rho = copy(density_matrix)

    egn_val = calc_Eigenvalues(rho)
    Norm = sum(egn_val)
    Purity = sum(egn_val.^2)

    #-----------------------------------
    return [Norm,Purity]
end

#   eigenvalue based generic properties of a density matrix
#-----------------------------------
function calc_SvN(density_matrix)
    rho = copy(density_matrix)

    egn_val = calc_Eigenvalues(rho)
    SvN = sum( [ - lam*log(lam) for lam in egn_val if lam > 0 ] )

    #-----------------------------------
    return SvN
end


calc_SvN (generic function with 1 method)

In [7]:
#   entanglement details between two sites
#-----------------------------------
function calc_Neg_2site(wf::MPS,locs)
    psi = copy(wf)
    A,B = locs

    rho = get_2pdm(psi,[A,B])
    old = inds(rho)

    new = [old[1],old[2],old[4],old[3]]
    rho_PT = swapinds(rho,old,new)
    egn_val = calc_Eigenvalues(rho_PT)

    Neg =  abs(sum( [ lam for lam in egn_val if lam<0 ]))
    #-----------------------------------
    return Neg
end



#   entanglement details between two sites
#-----------------------------------
function calc_MutInf_2site(wf::MPS,locs)
    psi = copy(wf)
    A,B = locs

    rho_A  = get_2pdm(psi,[A,A])
    rho_B  = get_2pdm(psi,[B,B])
    rho_AB = get_2pdm(psi,[A,B])

    S_A = calc_SvN(rho_A)
    S_B = calc_SvN(rho_B)
    S_AB = calc_SvN(rho_AB)

    MutInf = S_A + S_B - S_AB
    #-----------------------------------
    return MutInf
end

calc_MutInf_2site (generic function with 1 method)

In [8]:
#   entanglement details between four sites
#-----------------------------------
function calc_LogNeg_4site(wf::MPS,locs)    
    psi = copy(wf)
    A,C = locs

    rho = get_4pdm(psi,[A,C])
    old = inds(rho)

    function get_LogNeg(rho,old,new)
        rho_PT = swapinds(rho,old,new)
        egn_val = calc_Eigenvalues(rho_PT)
        Neg =  abs(sum( [lam for lam in egn_val if lam<0] ))
        return log(1+2*Neg)
    end
        
    #   ABCD = 1234 
    #   ABCD = 5678

    # wrt A B C D
    new_A  = [old[5],old[2],old[3],old[4],old[1],old[6],old[7],old[8]]
    new_B  = [old[1],old[6],old[3],old[4],old[5],old[2],old[7],old[8]]
    new_C  = [old[1],old[2],old[7],old[4],old[5],old[6],old[3],old[8]]
    new_D  = [old[1],old[2],old[3],old[8],old[5],old[6],old[7],old[4]]

    # wrt AB AC AD
    new_AB = [old[5],old[6],old[3],old[4],old[1],old[2],old[7],old[8]]
    new_AC = [old[5],old[2],old[7],old[4],old[1],old[6],old[3],old[8]]
    new_AD = [old[5],old[2],old[3],old[8],old[1],old[6],old[7],old[4]]

    LogNeg = 1
    for new in [new_A,new_B,new_C,new_D,new_AB,new_AC,new_AD]
        LogNeg *= get_LogNeg(rho,old,new)
    end
    LogNeg = LogNeg^(1.0/7.0)

    #-----------------------------------
    return LogNeg
end


calc_LogNeg_4site (generic function with 1 method)

In [9]:
#define the parameters of the hamiltonian and constuct MPO
#----------------------------------------------------------------------------

J = 1
B = 0.01

nsweeps = 20
maxdim = [10,25,50,100,150,200,250,300,400,500]
cutoff = [1E-11]

#--------------------
N = 30
#--------------------

site_list = collect(1:N) 
sites = siteinds( k->isodd(k) ? "S=1/2" : "S=1", N ) 
#sites = siteinds("S=1/2", N)

H = Hamiltonian(sites) 

# ground state and first excited state
#--------------------------------------------
energy0,psi0 = dmrg(H,random_mps(sites);nsweeps,maxdim,cutoff,ishermitian=true) ;
#energy1,psi1 = dmrg(H,[psi0],random_mps(sites);nsweeps,maxdim,cutoff,ishermitian=true) ;

After sweep 1 energy=-21.09995164081418  maxlinkdim=8 maxerr=3.91E-16 time=27.974
After sweep 2 energy=-21.524166113912518  maxlinkdim=25 maxerr=2.52E-07 time=0.141
After sweep 3 energy=-21.809258327372287  maxlinkdim=50 maxerr=3.21E-07 time=0.668
After sweep 4 energy=-21.847167358746436  maxlinkdim=100 maxerr=3.31E-08 time=3.116
After sweep 5 energy=-21.859141593662564  maxlinkdim=150 maxerr=8.48E-09 time=6.284
After sweep 6 energy=-21.866154793518362  maxlinkdim=200 maxerr=3.18E-09 time=14.870
After sweep 7 energy=-21.87150842921357  maxlinkdim=250 maxerr=8.16E-10 time=30.017
After sweep 8 energy=-21.875936373757256  maxlinkdim=300 maxerr=1.91E-10 time=45.760
After sweep 9 energy=-21.87930219419905  maxlinkdim=400 maxerr=2.66E-11 time=60.173
After sweep 10 energy=-21.881700931023733  maxlinkdim=500 maxerr=1.07E-11 time=87.353
After sweep 11 energy=-21.88335744602008  maxlinkdim=500 maxerr=1.32E-11 time=100.023
After sweep 12 energy=-21.884457130902526  maxlinkdim=500 maxerr=1.21E-11 

In [10]:
#* check thermodynamic limits *#
#-----------------------------

wf = Vector{Any}(undef,8)

wf[1] = MagnonOperation( psi0, [1] )         #one single magnon
wf[2] = MagnonOperation( psi0, [1,3] )       #two neughbouring
wf[3] = MagnonOperation( psi0, [1,3,5] )     #three neughbouring
wf[4] = MagnonOperation( psi0, [1,2] )       #two overlapping magnons 
wf[5] = MagnonOperation( psi0, [1,4] )       #two magnon separted by spin 1/2 
wf[6] = MagnonOperation( psi0, [2,5] )       #two magnon separted by spin 3/2
wf[7] = MagnonOperation( psi0, [1,5] )       #two magnon separed by two sites
wf[8] = MagnonOperation( psi0, [1,3]; split="yes" )       #split magnon of width 0

for idx in 1:length(wf)
    ovlap = abs(inner(wf[idx],psi0))
    print( f"{ovlap:e} \t " ) 
end

3.191369e-01 	 1.181095e-01 	 4.429155e-02 	 1.374915e-01 	 1.047707e-01 	 9.623538e-02 	 1.017893e-01 	 2.661131e-02 	 

In [11]:
import Pkg; Pkg.add("LaTeXStrings")

   Resolving package versions...
  No Changes to `~/.julia/environments/v1.10/Project.toml`
  No Changes to `~/.julia/environments/v1.10/Manifest.toml`


In [12]:
# 4 partitle negativity between (A,B,C,D) 
# inputs = A,C as B = A+1 and D=C+1
#------------------------------------------------------
using LaTeXStrings
A = 1
seps=0:5
enta= []
for sep in seps
    C =  A+2+sep
    ent = calc_LogNeg_4site(psi0,[A,C])
    push!(enta, ent)
    println(f"{sep} \t {ent}")
end

plot(seps, enta, xlabel= "Separation", ylabel=L"\mathcal{N}_4", label="1/2--1", marker=:circle, color="red", lw=2)


0 	 0.5548298031357911
1 	 0.28053648104640116
2 	 0.12929416974031765
3 	 0.09214229791082762
4 	 0.0936550064631928
5 	 0.08937694127165514


GKS: dlopen(/Users/Hank/.julia/artifacts/66ebf8a541da7166677a6e2ac6b8277592d32044/lib/cairoplugin.so, 1): Library not loaded: @rpath/libcairo.2.dylib
  Referenced from: /Users/Hank/.julia/artifacts/66ebf8a541da7166677a6e2ac6b8277592d32044/lib/cairoplugin.so
  Reason: Incompatible library version: cairoplugin.so requires version 11603.0.0 or later, but libcairo.2.dylib provides version 2.0.0


In [13]:
trinds1 = collect(3 : 4)
trinds2 = trinds1[end] .+ trinds1 .+ 2

2-element Vector{Int64}:
  9
 10

In [14]:
for sep in 0:3
    println(sep)
end

0
1
2
3
